In [ ]:
import scanpy as sc

In [ ]:
adata = sc.read_h5ad('../data/')

In [ ]:

# --- Find marker genes per cluster ---
sc.tl.rank_genes_groups(adata, groupby="clusters", method="t-test")
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

# get a tidy table of top markers
markers = sc.get.rank_genes_groups_df(adata, group=None)
markers.head()

In [ ]:
marker_genes = pd.DataFrame({
    group: adata.uns['rank_genes_groups']['names'][group][:5]
    for group in adata.uns['rank_genes_groups']['names'].dtype.names
})
marker_genes.head()
#marker_genes.to_csv('../data/broad_markers_leiden0-5.csv')

In [ ]:
from mllmcelltype import annotate_clusters, setup_logging

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # will read .env into environment
api_key = os.getenv("../OPENAI_API_KEY")

In [ ]:
marker_genes = marker_genes.T

In [ ]:
marker_genes = marker_genes.reset_index().rename(columns={'index':'cluster'})

In [ ]:
marker_genes = marker_genes.set_index("cluster").T.to_dict("list")

In [ ]:
import os

# Annotate clusters with a single model
annotations = annotate_clusters(
    marker_genes=marker_genes,  # DataFrame or dictionary of marker genes
    species='mouse',               # Organism species
    provider='openai',            # LLM provider
    model='gpt-4o-mini',               # Specific model
    tissue='brain'                #Tissue context (optional but recommended)
)

# Print annotations
for cluster, annotation in annotations.items():
    print(f"Cluster {cluster}: {annotation}")